select subject session FOV

In [2]:
import BCI_analysis, os
BCI_analysis.io_suite2p.get_canned_file_version()

'1.01'

In [5]:
# import numpy as np
# data = np.load(os.path.join(save_path,'BCI_58-070323.npy'),allow_pickle = True).tolist()

In [1]:
import BCI_analysis, os
local_temp_dir = '/home/jupyter/temp/' 
metadata_dir = '/home/jupyter/bucket/Metadata/' 
raw_data_path ='/home/jupyter/bucket/Data/Calcium_imaging/raw/Bergamo-2P-Photostim/' 
suite2p_path = '/home/jupyter/bucket/Data/Calcium_imaging/suite2p/Bergamo-2P-Photostim/'
behavior_data_path = '/home/jupyter/bucket/Data/Behavior/BCI_exported/Bergamo-2P-Photostim/'
setup = 'Bergamo-2P-Photostim'#'DOM3-MMIMS'
save_path = "/home/jupyter/bucket/Data/Calcium_imaging/canned_sessions"
fov = None#'FOV_01'
subject = ['BCI_58','BCI_59','BCI_60','BCI_64','BCI_65']#
BCI_analysis.io_suite2p.generate_canned_sessions(suite2p_path,
                                                 raw_data_path,
                                                 behavior_data_path,
                                                 save_path,
                                                 overwrite=False,
                                                 mouse_list=subject,
                                                 fov_list=fov, 
                                                 session_list=None)




No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally
['FOV_01', 'FOV_02']
FOV: FOV_01, Session Date: 070323


/home/jupyter/Scripts/BCI_analysis/BCI_analysis/io_bci/io_suite2p.py:484: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  closed_loop_scanimage_filenames = np.asarray(bpod_zaber_data["scanimage_file_names"])[files_with_movies]
/home/jupyter/Scripts/BCI_analysis/BCI_analysis/io_bci/io_suite2p.py:496: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dist_from_cn = np.asarray(dist_from_cn)[np.asarray(needed_cn_indices)]
/home/jupyter/Scripts/BCI_analysis/BCI_analysis/io_bci/io_suite2p.py:497: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

1 conditioned neuron(s) found: [72]
FOV: FOV_01, Session Date: 070523
2 conditioned neuron(s) found: [72 43]
FOV: FOV_01, Session Date: 070623
2 conditioned neuron(s) found: [ 43 212]
photostim exported
FOV: FOV_01, Session Date: 070723
2 conditioned neuron(s) found: [212 26]
photostim exported
FOV: FOV_01, Session Date: 071023
2 conditioned neuron(s) found: [26 51]
photostim exported
FOV: FOV_01, Session Date: 071123
3 conditioned neuron(s) found: [51 61 32]
photostim exported
FOV: FOV_01, Session Date: 071223
2 conditioned neuron(s) found: [ 32 119]
photostim exported
FOV: FOV_01, Session Date: 071323
2 conditioned neuron(s) found: [119  78]
photostim exported
FOV: FOV_02, Session Date: 071823
1 conditioned neuron(s) found: [37]
FOV: FOV_02, Session Date: 071923
multiple scanfields for dat_roi_44
rotating ROIs with -0.05163065224864266 degrees
multiple scanfields for dat_roi_2
rotating ROIs with -0.05163065224864266 degrees
multiple scanfields for dat_roi_129
rotating ROIs with -0.05

/home/jupyter/Scripts/BCI_analysis/BCI_analysis/io_bci/io_suite2p.py:487: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if clt_ in closed_loop_scanimage_filenames:


photostim exported
FOV: FOV_02, Session Date: 072323
1 conditioned neuron(s) found: [39]
photostim exported
FOV: FOV_02, Session Date: 072423
1 conditioned neuron(s) found: [150]
photostim exported
['FOV_01', 'FOV_02', 'FOV_03']
FOV: FOV_01, Session Date: 070323
1 conditioned neuron(s) found: [22]
FOV: FOV_01, Session Date: 070523
2 conditioned neuron(s) found: [22  3]
FOV: FOV_01, Session Date: 070623
2 conditioned neuron(s) found: [ 3 95]
photostim exported
FOV: FOV_01, Session Date: 070723
multiple movies for a behavior trial, skipped: ['neuron29_00001.tif' 'neuron29_00002.tif']
2 conditioned neuron(s) found: [95 28]
photostim exported
FOV: FOV_02, Session Date: 071123
1 conditioned neuron(s) found: [254]
FOV: FOV_02, Session Date: 071223
2 conditioned neuron(s) found: [254 115]
photostim exported
FOV: FOV_02, Session Date: 071323
2 conditioned neuron(s) found: [115  14]
photostim exported
FOV: FOV_03, Session Date: 071823
1 conditioned neuron(s) found: [5]
FOV: FOV_03, Session Date

/opt/conda/envs/bci_with_suite2p/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/envs/bci_with_suite2p/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/conda/envs/bci_with_suite2p/lib/python3.8/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/envs/bci_with_suite2p/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/opt/conda/envs/bci_with_suite2p/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


FOV: FOV_02, Session Date: 072323
1 conditioned neuron(s) found: [208]
photostim exported
FOV: FOV_02, Session Date: 072423
multiple movies for a behavior trial, skipped: ['neuron256_00001.tif' 'neuron256_00002.tif']
1 conditioned neuron(s) found: [255]
photostim exported
['FOV_01']
FOV: FOV_01, Session Date: 072123
1 conditioned neuron(s) found: [2]
FOV: FOV_01, Session Date: 072223
1 conditioned neuron(s) found: [18]
photostim exported
FOV: FOV_01, Session Date: 072323
1 conditioned neuron(s) found: [63]
photostim exported
['FOV_01']
FOV: FOV_01, Session Date: 072123
2 conditioned neuron(s) found: [25 29]
FOV: FOV_01, Session Date: 072223
1 conditioned neuron(s) found: [31]
photostim exported
FOV: FOV_01, Session Date: 072323
1 conditioned neuron(s) found: [10]
photostim exported


In [2]:
import pandas as pd
metadata = {'filename':[],
           'mouse':[],
           'session':[],
           'fov':[],
           'neuron number':[],
           'spontaneous length (s)':[],
           'BCI_1 CN':[],
           'BCI_1 trial number':[],
           'BCI_1 hit rate':[],
           'BCI_2 CN':[],
           'BCI_2 trial number':[],
           'BCI_2 hit rate':[],
           'photostim group number':[],
           'photostim repeats per group':[]}
#generate metadata table
import numpy as np
files = os.listdir(save_path)
for file in files:
    if '.npy' not in file:
        continue
    data_dict = np.load(os.path.join(save_path,file),allow_pickle = True).tolist()
    if 'spont' in data_dict.keys():
        metadata['spontaneous length (s)'].append(int(data_dict['spont']['Ftrace'].shape[1] * data_dict['dt_si']))
    else:
        metadata['spontaneous length (s)'].append(0)
    metadata['filename'].append(file)
    metadata['mouse'].append(data_dict['mouse'])
    metadata['session'].append(data_dict['session'])
    metadata['fov'].append(data_dict['fov'])
    metadata['neuron number'].append(len(data_dict['iscell']))
    for k in ['BCI_1','BCI_2']:#,'BCI_3','BCI_4']:
        if k not in data_dict.keys():
            metadata['{} CN'.format(k)].append(None)
            metadata['{} trial number'.format(k)].append(None)
            metadata['{} hit rate'.format(k)].append(None)
        else:
            metadata['{} CN'.format(k)].append(data_dict[k]['conditioned_neuron'])
            metadata['{} trial number'.format(k)].append(data_dict[k]['F'].shape[2])
            metadata['{} hit rate'.format(k)].append(np.round(sum(data_dict[k]['reward_time'])/data_dict[k]['F'].shape[2],2))
    if 'photostim' in data_dict.keys():
        metadata['photostim group number'].append(data_dict['photostim']['favg'].shape[2])
        metadata['photostim repeats per group'].append(data_dict['photostim']['Fstim'].shape[2]/data_dict['photostim']['favg'].shape[2])
    else:
        metadata['photostim group number'].append(None)
        metadata['photostim repeats per group'].append(None)
df = pd.DataFrame(metadata)
df.to_csv(os.path.join(save_path,'metadata.csv'))

In [ ]:
##### old canning
# import BCI_analysis, os
# local_temp_dir = '/home/jupyter/temp/' 
# metadata_dir = '/home/jupyter/bucket/Metadata/' 
# raw_scanimage_dir_base ='/home/jupyter/bucket/Data/Calcium_imaging/raw/' 
# suite2p_dir_base = '/home/jupyter/bucket/Data/Calcium_imaging/suite2p/'
# bpod_path = '/home/jupyter/bucket/Data/Behavior/BCI_exported/'
# setup = 'Bergamo-2P-Photostim'#'DOM3-MMIMS'
# save_path = "/home/jupyter/bucket/Data/Calcium_imaging/sessionwise_tba"
# fov = 'FOV_01'
# subject = 'BCI_66'
# BCI_analysis.io_suite2p.suite2p_to_npy(os.path.join(suite2p_dir_base,setup), 
#                                        os.path.join(raw_scanimage_dir_base,setup), 
#                                        os.path.join(bpod_path,setup),
#                                        save_path, 
#                                        overwrite=False, 
#                                        mice_name = subject,
#                                        fov_list = [fov],
#                                        session_list=None,#['072323'],
#                                        max_frames = 500)

FOV: FOV_01, Session Date: .ipynb_checkpoints
Session already exists at /home/jupyter/bucket/Data/Calcium_imaging/sessionwise_tba/BCI_66/BCI_66-092523-FOV_01.npy, and overwrite=False


In [4]:
# #register Z-stack
# import os
# from pathlib import Path
# subject = 'PKJ_04'
# session = '042123'
# z_stack = 'stack_00001.tif'
# os.chdir('/home/jupyter/Scripts/Suite2p_pipeline/utils/')
# import utils_imaging as utils_imaging
# local_temp_dir = '/home/jupyter/temp'
# temp_dir = os.path.join(local_temp_dir,'{}_{}_{}'.format(subject,session,z_stack[:-4]))
# Path(temp_dir).mkdir(exist_ok = True, parents = True)
# utils_imaging.register_zstack('/home/jupyter/bucket/Data/Calcium_imaging/raw/Bergamo-2P-Photostim/{}/{}/{}'.format(subject,session,z_stack) ,temp_dir)

planes: 51
(51, 100, 2, 800, 800)
(5100, 2, 800, 800)
pixel size changed from [5.4, 5.4] to 0.45442069540907865 
regstering /home/jupyter/temp/PKJ_04_042123_stack_00001/stack_00001.tif
{}
tif
** Found 1 tifs - converting to binary **
408 frames of binary, time 2.05 sec.
816 frames of binary, time 3.94 sec.
1224 frames of binary, time 5.14 sec.
1632 frames of binary, time 6.43 sec.
2040 frames of binary, time 8.59 sec.
2448 frames of binary, time 11.18 sec.
2856 frames of binary, time 13.77 sec.
3264 frames of binary, time 16.45 sec.
3672 frames of binary, time 20.01 sec.
4080 frames of binary, time 24.26 sec.
4488 frames of binary, time 28.65 sec.
4896 frames of binary, time 47.67 sec.
5304 frames of binary, time 75.08 sec.
5712 frames of binary, time 95.12 sec.
6120 frames of binary, time 117.35 sec.
6528 frames of binary, time 139.37 sec.
6936 frames of binary, time 161.87 sec.
7344 frames of binary, time 181.85 sec.
7752 frames of binary, time 203.13 sec.
8160 frames of binary, time